In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Character information
Includes base stats, gear stats and raid buffs.

In [ ]:
#base stats
base_int = 95
gear_int = 188
base_spi = 95
gear_spi = 0

#spell improvements
arcane_sp = 1056
nature_sp = 834
crit = 0.2072
hit = 0.99

#consumables
sp_flask = 80
food_sp = 23
food_spi = 20
wizard_oil = 36
mana_pot_min = 1800
mana_pot_max = 3000
mana_pot_cd = 120

#raid buffs
arcane_brilliance = 40
prayer_of_spirit = 50
improved_spirit_sp = 0.1
gift_of_wild = 14 * 1.35
blessing_of_kings = 1.1
blessing_of_wisdom = 41 * 1.2
moonkin_aura = 0.05

#total stats
total_int = (base_int + gear_int + arcane_brilliance + gift_of_wild) * blessing_of_kings
total_spi = (base_spi + gear_spi + prayer_of_spirit + gift_of_wild + food_spi) * blessing_of_kings
total_arc_sp = arcane_sp + sp_flask + food_sp + wizard_oil + 0.25 * total_int
total_nat_sp = nature_sp + sp_flask + food_sp + wizard_oil + 0.25 * total_int
total_crit = crit + moonkin_aura #missing critical information q:-D

#mana regen
gear_mp5 = 46
spirit_mp5 = 5 * (0.001 + math.sqrt(total_int) * total_spi * 0.005596)
dreamstate = total_int / 10
mp5_casting_total = gear_mp5 + spirit_mp5 * 0.3 + dreamstate

#mana pool
base_mana = 2370
mana_pool = base_mana + (20 + (15 * (total_int - 20)))

# Spell information
Mana costs, casting times, crit chances and durations.

In [ ]:
#starfire
sf_cost = 370 * 0.91
sf_cast = 3.0
sf_crit = crit + 0.04
sf_min_dmg = (550 + (total_arc_sp + 55) * 1.2) * 1.1
sf_max_dmg = (647 + (total_arc_sp + 55) * 1.2) * 1.1

#wrath
wrath_cost = 255 * 0.91
wrath_cast = 1.5
wrath_crit = crit + 0.04

#moonfire
mf_cost = 495 * 0.91
mf_cast = 1.5
mf_crit = crit + 0.1
mf_duration = 12
mf_min_dmg = (305 + total_arc_sp * 0.1495) * 1.1 * 1.1
mf_max_dmg = (357 + total_arc_sp * 0.1495) * 1.1 * 1.1
mf_dot_dmg = (150 + total_arc_sp * 0.1302) * 1.1 * 1.1 * 4

#insect swarm
is_cost = 175
is_cast = 1.5
is_duration = 12
is_dot_dmg = 132 + total_nat_sp * 0.2 * 6

#faerie fire
ff_cost = 145
ff_cast = 1.5
ff_duration = 40

#innervate
inner_cost = base_mana * 0.04
inner_duration = 20
inner_cd = 360

# Spell casts
Every spell has it's own function which calculates damage dealt and time spent casting. Casting functions return all relevant information to main rotation function.

In [ ]:
#starfire
def starfire(natures_grace):
    #nature's grace reduces casting time by 0.5 sec
    cast_time = sf_cast
    if natures_grace:
        cast_time = sf_cast - 0.5
    #calculate non-critical damage dealt
    damage = np.random.randint(sf_min_dmg, sf_max_dmg + 1)
    #crit chance
    critted = np.random.binomial(1, total_crit)
    if critted:
        damage = damage * 2
    #return final damage dealt, crit info and time spent casting
    return(damage, critted, cast_time)

#moonfire
def moonfire(natures_grace):    
    #calculate non-critical damage dealt
    damage = np.random.randint(mf_min_dmg, mf_max_dmg + 1)
    #crit chance
    critted = np.random.binomial(1, total_crit)
    if critted:
        damage = damage * 2
    #nature's grace is not consumed by moonfire so "crit" is set to 1
    if natures_grace:
        critted = 1
    damage = damage + mf_dot_dmg
    #return final damage dealt, crit info and time spent casting
    return(round(damage), critted)

#insect swarm
def insect_swarm(natures_grace):    
    damage = is_dot_dmg
    #return damage
    return(round(damage))

# Rotation priorities
Goal is to create functions for choosing which spell to cast. All rotations will prioritize keeping Faerie Fire up.
### Rotations
1. Mana intense, high damage: keep Moonfire up and cast Starfire
2. Balanced mana, balanced damage: keep Moonfire and Insect Swarm up, cast Starfire
3. Mana conserving, low damage: keep Insect Swarm up, cast Starfire

## Starfire + moonfire

In [ ]:
#fight duration and damage counters
fight_length = 0
total_damage = 0
damage = 0
mana_left = mana_pool
mana_timer = 5
cast_time = 0
natures_grace = 0
#debuffs and cooldowns
ff_dur_left = 0
mf_dur_left = 0
inner_dur_left = 0
inner_cd_left = 0
mana_pot_cd_left = 0
#spell counters
starfires = 0
moonfires = 0
faerie_fires = 0
mana_pots = 0
innervates = 0

while mana_left >= mf_cost:
    #reset single spell dmg for each cycle
    damage = 0
    #cast faerie fire if not active
    if ff_dur_left < 1:
        mana_left -= ff_cost
        cast_time = ff_cast
        ff_dur_left = ff_duration
        faerie_fires += 1
    #cast moonfire if not active
    elif mf_dur_left < 1:
        mana_left -= mf_cost
        cast_time = mf_cast
        mf_dur_left = mf_duration
        damage, natures_grace = moonfire(natures_grace)
        moonfires += 1
    #cast starfire if both debuffs active
    else:
        mana_left -= sf_cost
        damage, natures_grace, cast_time = starfire(natures_grace)
        starfires += 1
    
    #update timers after casting
    fight_length += cast_time
    ff_dur_left -= cast_time
    mf_dur_left -= cast_time
    inner_dur_left -= cast_time
    mana_pot_cd_left -= cast_time
    
    #regen mana if threshold passed
    if fight_length >= mana_timer:
        #increase if innervate active
        if inner_dur_left > 0:
            mana_left += spirit_mp5 * 5
        else:
            mana_left += spirit_mp5 * 0.3
        mana_left += gear_mp5 + dreamstate
        mana_timer += 5
    
    #use mana pot if enough mana spent and not on cd
    if mana_pool - mana_left > 3000 and mana_pot_cd_left < 1:
        mana_left += np.random.randint(mana_pot_min, mana_pot_max + 1)
        mana_pot_cd_left = mana_pot_cd
        mana_pots += 1
    
    #use innervate if mana low and off cd
    if mana_left / mana_pool < 0.25 and inner_cd_left < 1:
        inner_dur_left = inner_duration
        inner_cd_left = inner_cd
        innervates += 1
    
    #update total damage
    total_damage += damage

print("moonfires: " + str(moonfires))
print("starfires: " + str(starfires))
print("faerie fires: " + str(faerie_fires))
print("mana pots: " + str(mana_pots))
print("innervates: " + str(innervates))
print("total damage: " + str(total_damage))
print("fight duration: " + str(fight_length))
print("dps: " + str(total_damage/fight_length))

## Starfire + Insect Swarm + Moonfire

In [ ]:
#fight duration and damage counters
fight_length = 0
total_damage = 0
damage = 0
mana_left = mana_pool
mana_timer = 5
cast_time = 0
natures_grace = 0
#debuffs and cooldowns
ff_dur_left = 0
mf_dur_left = 0
is_dur_left = 0
inner_dur_left = 0
inner_cd_left = 0
mana_pot_cd_left = 0
#spell counters
starfires = 0
moonfires = 0
insect_swarms = 0
faerie_fires = 0
mana_pots = 0
innervates = 0

while mana_left >= mf_cost:
    #reset single spell dmg for each cycle
    damage = 0
    #cast faerie fire if not active
    if ff_dur_left < 1:
        mana_left -= ff_cost
        cast_time = ff_cast
        ff_dur_left = ff_duration
        faerie_fires += 1
    #cast moonfire if not active
    elif mf_dur_left < 1:
        mana_left -= mf_cost
        cast_time = mf_cast
        mf_dur_left = mf_duration
        damage, natures_grace = moonfire(natures_grace)
        moonfires += 1
    #cast insect swarm if not active
    elif is_dur_left < 1:
        mana_left -= is_cost
        cast_time = is_cast
        damage = round(is_dot_dmg)
        is_dur_left = is_duration
        insect_swarms += 1
    #cast starfire if both debuffs active
    else:
        mana_left -= sf_cost
        damage, natures_grace, cast_time = starfire(natures_grace)
        starfires += 1
    
    #update timers after casting
    fight_length += cast_time
    ff_dur_left -= cast_time
    mf_dur_left -= cast_time
    is_dur_left -= cast_time
    inner_dur_left -= cast_time
    mana_pot_cd_left -= cast_time
    
    #regen mana if threshold passed
    if fight_length >= mana_timer:
        #increase if innervate active
        if inner_dur_left > 0:
            mana_left += spirit_mp5 * 5
        else:
            mana_left += spirit_mp5 * 0.3
        mana_left += gear_mp5 + dreamstate
        mana_timer += 5
    
    #use mana pot if enough mana spent and not on cd
    if mana_pool - mana_left > 3000 and mana_pot_cd_left < 1:
        mana_left += np.random.randint(mana_pot_min, mana_pot_max + 1)
        mana_pot_cd_left = mana_pot_cd
        mana_pots += 1
    
    #use innervate if mana low and off cd
    if mana_left / mana_pool < 0.25 and inner_cd_left < 1:
        inner_dur_left = inner_duration
        inner_cd_left = inner_cd
        innervates += 1
    
    #update total damage
    total_damage += damage

print("moonfires: " + str(moonfires))
print("insect swarms: " + str(insect_swarms))
print("starfires: " + str(starfires))
print("faerie fires: " + str(faerie_fires))
print("mana pots: " + str(mana_pots))
print("innervates: " + str(innervates))
print("total damage: " + str(total_damage))
print("fight duration: " + str(fight_length))
print("dps: " + str(total_damage/fight_length))

## Starfire + Insect Swarm

In [ ]:
#fight duration and damage counters
fight_length = 0
total_damage = 0
damage = 0
mana_left = mana_pool
mana_timer = 5
cast_time = 0
natures_grace = 0
#debuffs and cooldowns
ff_dur_left = 0
is_dur_left = 0
inner_dur_left = 0
inner_cd_left = 0
mana_pot_cd_left = 0
#spell counters
starfires = 0
insect_swarms = 0
faerie_fires = 0
mana_pots = 0
innervates = 0

while mana_left >= mf_cost:
    #reset single spell dmg for each cycle
    damage = 0
    #cast faerie fire if not active
    if ff_dur_left < 1:
        mana_left -= ff_cost
        cast_time = ff_cast
        ff_dur_left = ff_duration
        faerie_fires += 1
    elif is_dur_left < 1:
        mana_left -= is_cost
        cast_time = is_cast
        damage = round(is_dot_dmg)
        is_dur_left = is_duration
        insect_swarms += 1
    #cast starfire if both debuffs active
    else:
        mana_left -= sf_cost
        damage, natures_grace, cast_time = starfire(natures_grace)
        starfires += 1
    
    #update timers after casting
    fight_length += cast_time
    ff_dur_left -= cast_time
    is_dur_left -= cast_time
    inner_dur_left -= cast_time
    mana_pot_cd_left -= cast_time
    
    #regen mana if threshold passed
    if fight_length >= mana_timer:
        #increase if innervate active
        if inner_dur_left > 0:
            mana_left += spirit_mp5 * 5
        else:
            mana_left += spirit_mp5 * 0.3
        mana_left += gear_mp5 + dreamstate
        mana_timer += 5
    
    #use mana pot if enough mana spent and not on cd
    if mana_pool - mana_left > 3000 and mana_pot_cd_left < 1:
        mana_left += np.random.randint(mana_pot_min, mana_pot_max + 1)
        mana_pot_cd_left = mana_pot_cd
        mana_pots += 1
    
    #use innervate if mana low and off cd
    if mana_left / mana_pool < 0.25 and inner_cd_left < 1:
        inner_dur_left = inner_duration
        inner_cd_left = inner_cd
        innervates += 1
    
    #update total damage
    total_damage += damage

print("insect swarms: " + str(insect_swarms))
print("starfires: " + str(starfires))
print("faerie fires: " + str(faerie_fires))
print("mana pots: " + str(mana_pots))
print("innervates: " + str(innervates))
print("total damage: " + str(total_damage))
print("fight duration: " + str(fight_length))
print("dps: " + str(total_damage/fight_length))